In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas import json_normalize


# Display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)  # Adjust as needed
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Data path
json_path = "../data/raw/credit_report_sample.json"

In [2]:
with open(json_path, 'r') as f:
    json_data = json.load(f)

if isinstance(json_data, list):
    df_json = pd.DataFrame(json_data)
    df_json.info()
else:
    print("dictionary JSON structure:")
    print(json.dumps(json_data, indent=2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   application_id  3 non-null      int64 
 1   data            3 non-null      object
dtypes: int64(1), object(1)
memory usage: 180.0+ bytes


In [4]:
print('Size of sample Json file:',df_json.shape)
df_json.head(1)

Size of sample Json file: (3, 2)


application_id  \
0         9711360   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

### First Level clean to Understand vars

In [17]:
# Extract just the top-level keys inside `data.consumerfullcredit`
def extract_top_keys(row):
    return list(row.get('data', {}).get('consumerfullcredit', {}).keys())

# Apply to all rows and get a union of all unique keys
top_keys = set()
for record in df_json.to_dict(orient='records'):
    top_keys.update(extract_top_keys(record))

top_keys = ['application_id'] + sorted(top_keys)
# print("Expected top-level variables:", top_keys)

# Create a flattened DataFrame with just top-level sections
flattened_rows = []
for record in df_json.to_dict(orient='records'):
    row = {'application_id': record['application_id']}
    consumer_credit = record.get('data', {}).get('consumerfullcredit', {})
    for key in top_keys[1:]:  # skip application_id
        row[key] = consumer_credit.get(key)
    flattened_rows.append(row)


df_level1_flat = pd.DataFrame(flattened_rows)
print('Top Level categories of Vars:',df_level1_flat.shape)

df_level1_flat.columns

Top Level categories of Vars: (3, 16)


Index(['application_id', 'accountmonthlypaymenthistory',
       'accountmonthlypaymenthistoryheader', 'accountrating',
       'creditaccountsummary', 'creditagreementsummary',
       'deliquencyinformation', 'employmenthistory', 'enquirydetails',
       'enquiryhistorytop', 'guarantorcount', 'guarantordetails',
       'identificationhistory', 'personaldetailssummary', 'subjectlist',
       'telephonehistory'],
      dtype='object')

In [18]:
df_level1_flat.head()

application_id  \
0         9711360   
1         9714953   
2         9714978   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [38]:
# var_expand = pd.json_normalize(df_level1_flat['deliquencyinformation'])
# var_expand.head()

,accountno,periodnum,subscribername,monthsinarrears
0,2150224012,20140930,First City Monument Bank Ltd Lagos,13
1,XXX,20170430,First City Monument Bank Ltd Lagos,2
2,4181042465,20200630,Branch International Financial Services Limited,109


## 1. Delinquincy

Numeric Variables

total_delinquency_accounts → Count of delinquent accounts linked to the applicant

months_in_arrears → Maximum number of months any account is in arrears

latest_delinquency_period → Latest delinquency period (numeric date format YYYYMMDD)

unique_delinquency_subscribers → Count of unique lenders/subscribers with delinquent accounts

Flags (Risk Indicators)

has_delinquency_flag → 1 if total_delinquency_accounts > 0

recent_delinquency_flag → 1 if latest_delinquency_period falls within last 6 months

severe_delinquency_flag → 1 if months_in_arrears ≥ 6

In [119]:
def extract_delinquency_features(df, col='deliquencyinformation', add_flags=True):
    # Normalize nested data
    records = []
    for _, row in df.iterrows():
        app_id = row['application_id']
        delinq_data = row[col]

        if isinstance(delinq_data, list):
            for item in delinq_data:
                item['application_id'] = app_id
                records.append(item)
        elif isinstance(delinq_data, dict):
            delinq_data['application_id'] = app_id
            records.append(delinq_data)

    if not records:
        return pd.DataFrame(columns=['application_id'])  # Empty case

    delinq_normalized = pd.DataFrame(records)

    # Replace "XXX" with NaN and convert numeric fields
    delinq_normalized.replace("XXX", np.nan, inplace=True)
    delinq_normalized['monthsinarrears'] = pd.to_numeric(delinq_normalized['monthsinarrears'], errors='coerce')
    delinq_normalized['periodnum'] = pd.to_numeric(delinq_normalized['periodnum'], errors='coerce')

    # Aggregate
    df_delinquency_features = delinq_normalized.groupby('application_id').agg(
        total_delinquency_accounts=('accountno', 'count'),
        months_in_arrears=('monthsinarrears', 'max'),
        latest_delinquency_period=('periodnum', 'max'),
        unique_delinquency_subscribers=('subscribername', 'nunique')
    ).reset_index()

    # Add optional flags
    if add_flags:
        df_delinquency_features['has_delinquency_flag'] = (df_delinquency_features['total_delinquency_accounts'] > 0).astype(int)
        df_delinquency_features['severe_delinquency_flag'] = (df_delinquency_features['months_in_arrears'] >= 6).astype(int)

        # Recent delinquency flag: check if latest period within last 6 months
        current_period = int(datetime.today().strftime("%Y%m"))
        six_months_ago = current_period - 6
        df_delinquency_features['recent_delinquency_flag'] = (
            df_delinquency_features['latest_delinquency_period'].fillna(0).astype(int) >= six_months_ago
        ).astype(int)

    # Return with application_id
    return pd.concat([df[['application_id']], df_delinquency_features.drop('application_id', axis=1)], axis=1)

In [120]:
df_deliquencyinformation_features = extract_delinquency_features(df_level1_flat)
df_deliquencyinformation_features.head()

,application_id,total_delinquency_accounts,months_in_arrears,latest_delinquency_period,unique_delinquency_subscribers,has_delinquency_flag,severe_delinquency_flag,recent_delinquency_flag
0,9711360,1,13,20140930,1,1,1,1
1,9714953,0,2,20170430,1,0,0,1
2,9714978,1,109,20200630,1,1,1,1


## 2. personaldetailssummary

- `age` → Age of applicant (calculated from birthdate)
- `dependants` → Number of dependents

- `gender` → Male/Female
- `nationality` → Country name (e.g., Nigeria)
- `employerdetail` → Employer details (e.g., ALL MILITARY STAFFS)
- `residential_state` → Extracted from residential address


In [121]:
def extract_personal_details_features(df, col='personaldetailssummary'):
    records = []
    for _, row in df.iterrows():
        app_id = row['application_id']
        details = row[col]

        if isinstance(details, dict):
            record = {
                'application_id': app_id,
                'gender': details.get('gender'),
                'birthdate': details.get('birthdate'),
                'dependants': details.get('dependants'),
                'nationality': details.get('nationality'),
                'employerdetail': details.get('employerdetail'),
                'residential_state': details.get('residentialaddress2')  # Extract state from address
            }
            records.append(record)

    if not records:
        return pd.DataFrame(columns=['application_id'])

    # Create DataFrame
    personal_details = pd.DataFrame(records)

    # Convert birthdate to datetime and calculate age
    personal_details['birthdate'] = pd.to_datetime(personal_details['birthdate'], format="%d/%m/%Y", errors='coerce')
    today = pd.Timestamp.today()
    personal_details['age'] = personal_details['birthdate'].apply(lambda x: today.year - x.year if pd.notnull(x) else np.nan)

    # Convert dependants to numeric
    personal_details['dependants'] = pd.to_numeric(personal_details['dependants'], errors='coerce')

    # Drop raw birthdate column (optional)
    personal_details.drop(columns=['birthdate'], inplace=True)

    return personal_details

In [122]:
df_personal_features = extract_personal_details_features(df_level1_flat, col='personaldetailssummary')
df_personal_features.head()

,application_id,gender,dependants,nationality,employerdetail,residential_state,age
0,9711360,Male,0,Nigeria,ALL MILITARY STAFFS,KADUNA STATE NIGERIA,34
1,9714953,Female,0,Nigeria,None,UMUAHIA,40
2,9714978,Female,0,Nigeria,None,Nigeria,42


## 3. Guarantor Details

- `guarantor_age` → Age of the guarantor
- `guarantor_count` → Total number of guarantors

- `has_guarantor` → 1 if guarantor_count > 0, else 0


In [123]:
import pandas as pd
import numpy as np
from datetime import datetime

def extract_guarantor_features(df, details_col='guarantordetails', count_col='guarantorcount'):
    """
    Extract guarantor-related features from nested data.

    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame with application_id and guarantor details.
    details_col : str
        Column name containing guarantor details dictionary.
    count_col : str
        Column name containing guarantor count dictionary.

    Returns:
    --------
    pd.DataFrame
        DataFrame with application_id and extracted guarantor features.
    """

    # ---- Normalize guarantor details ----
    guarantor_details = []
    for _, row in df.iterrows():
        app_id = row['application_id']
        details = row.get(details_col, {})
        if isinstance(details, dict):
            dob = details.get('guarantordateofbirth')
            if dob == '1900-01-01T00:00:00+01:00':
                dob = None
            guarantor_details.append({
                'application_id': app_id,
                'guarantordateofbirth': dob
            })
        else:
            guarantor_details.append({'application_id': app_id, 'guarantordateofbirth': None})

    guarantor_details_df = pd.DataFrame(guarantor_details)

    # Convert DOB to datetime and calculate age
    guarantor_details_df['guarantordateofbirth'] = pd.to_datetime(
        guarantor_details_df['guarantordateofbirth'], errors='coerce'
    )
    today = pd.to_datetime(datetime.today().date())
    guarantor_details_df['guarantor_age'] = (
        (today - guarantor_details_df['guarantordateofbirth']).dt.days // 365
    )

    # ---- Extract guarantor count ----
    guarantor_count_list = []
    for _, row in df.iterrows():
        app_id = row['application_id']
        count_info = row.get(count_col, {})
        count = None
        if isinstance(count_info, dict):
            count = pd.to_numeric(count_info.get('guarantorssecured'), errors='coerce')
        guarantor_count_list.append({'application_id': app_id, 'guarantor_count': count})

    guarantor_count_df = pd.DataFrame(guarantor_count_list)

    # ---- Merge and create flags ----
    result = guarantor_details_df.merge(guarantor_count_df, on='application_id', how='left')
    result['has_guarantor'] = result['guarantor_count'].apply(
        lambda x: 1 if pd.notna(x) and x > 0 else 0
    )

    return result

In [124]:
df_guarantor_features = extract_guarantor_features(df_level1_flat, details_col='guarantordetails', count_col='guarantorcount')
df_guarantor_features.head()

,application_id,guarantordateofbirth,guarantor_age,guarantor_count,has_guarantor
0,9711360,NaT,NaN,0,0
1,9714953,NaT,NaN,0,0
2,9714978,NaT,NaN,0,0


## 4. Account monthly payment history

| Feature Name                 | Description                              |
| ---------------------------- | ---------------------------------------- |
| `dpd_3m`                     | Max DPD in last 3 months                 |
| `dpd_6m`                     | Max DPD in last 6 months                 |
| `dpd_12m`                    | Max DPD in last 12 months                |
| `dpd_24m`                    | Max DPD in last 24 months                |
| `late_payments_12m_count`    | Count of late payments in last 12 months |
| `num_performing_accounts`    | Number of performing accounts            |
| `num_nonperforming_accounts` | Number of non-performing accounts        |
| `has_written_off_account`    | 1 if any account WrittenOff, else 0      |
| `num_open_accounts`          | Count of open accounts                   |
| `num_new_accounts_6m`        | Accounts opened in last 6 months         |
| `avg_utilization_ratio`      | Avg(current\_balance / opening\_balance) |
| `total_amount_overdue`       | Total overdue amount                     |
| `avg_account_age_months`     | Average age of accounts in months        |
| `max_account_age_months`     | Maximum account age in months            |


In [125]:
def extract_normalized_credit_features(df, history_column='accountmonthlypaymenthistory'):
    dpd_months = {
        "dpd_3m": [f"m{str(i).zfill(2)}" for i in range(22, 25)],
        "dpd_6m": [f"m{str(i).zfill(2)}" for i in range(19, 25)],
        "dpd_12m": [f"m{str(i).zfill(2)}" for i in range(13, 25)],
        "dpd_24m": [f"m{str(i).zfill(2)}" for i in range(1, 25)],
    }

    today = datetime.today()

    def clean_account(account):
        # Clean DPD month fields
        for i in range(1, 25):
            m_key = f"m{str(i).zfill(2)}"
            val = account.get(m_key, None)
            if val in ['XXX', '#']:
                account[m_key] = np.nan
            else:
                try:
                    account[m_key] = int(val)
                except:
                    account[m_key] = np.nan

        # Clean numeric fields
        for amt_field in ['currentbalanceamt', 'openingbalanceamt', 'amountoverdue']:
            val = account.get(amt_field)
            try:
                account[amt_field] = float(str(val).replace(",", ""))
            except:
                account[amt_field] = np.nan

        # Handle invalid date
        if account.get("dateaccountopened") == "1900-01-01":
            account["dateaccountopened"] = None

        return account

    def safe_float(val):
        try:
            return float(str(val).replace(",", ""))
        except:
            return np.nan

    def row_feature_extraction(row):
        # Normalize accounts
        accounts = row.get(history_column, [])
        accounts = [clean_account(acc) for acc in accounts if isinstance(acc, dict)]

        dpd_values = {k: [] for k in dpd_months}
        late_payments_12m = 0
        num_performing = 0
        num_nonperforming = 0
        has_written_off = False
        open_accounts = 0
        new_accounts_6m = 0
        util_ratios = []
        total_amount_overdue = 0
        account_ages = []

        for acc in accounts:
            perf_status = acc.get("performancestatus", "")
            acc_status = acc.get("accountstatus", "")
            date_opened_str = acc.get("dateaccountopened", None)

            if perf_status == "Performing":
                num_performing += 1
            else:
                num_nonperforming += 1

            if acc_status == "WrittenOff":
                has_written_off = True

            if acc_status == "Open":
                open_accounts += 1

            if date_opened_str:
                try:
                    date_opened = datetime.strptime(date_opened_str, "%d/%m/%Y")
                    acc_age_months = (today.year - date_opened.year) * 12 + (today.month - date_opened.month)
                    account_ages.append(acc_age_months)
                    if acc_age_months <= 6:
                        new_accounts_6m += 1
                except:
                    pass

            curr_bal = safe_float(acc.get("currentbalanceamt"))
            open_bal = safe_float(acc.get("openingbalanceamt"))
            if curr_bal is not None and open_bal is not None and open_bal > 0:
                util_ratios.append(curr_bal / open_bal)

            overdue = safe_float(acc.get("amountoverdue"))
            if overdue:
                total_amount_overdue += overdue

            for k, months in dpd_months.items():
                for m in months:
                    val = acc.get(m, '#')
                    try:
                        dpd_val = int(val)
                        dpd_values[k].append(dpd_val)
                        if k == 'dpd_12m' and dpd_val > 0:
                            late_payments_12m += 1
                    except (ValueError, TypeError):
                        continue

        return pd.Series({
            "dpd_3m": max(dpd_values["dpd_3m"], default=0),
            "dpd_6m": max(dpd_values["dpd_6m"], default=0),
            "dpd_12m": max(dpd_values["dpd_12m"], default=0),
            "dpd_24m": max(dpd_values["dpd_24m"], default=0),
            "late_payments_12m_count": late_payments_12m,
            "num_performing_accounts": num_performing,
            "num_nonperforming_accounts": num_nonperforming,
            "has_written_off_account": int(has_written_off),
            "num_open_accounts": open_accounts,
            "num_new_accounts_6m": new_accounts_6m,
            "avg_utilization_ratio": np.nanmean(util_ratios) if util_ratios else np.nan,
            "total_amount_overdue": total_amount_overdue,
            "avg_account_age_months": np.mean(account_ages) if account_ages else np.nan,
            "max_account_age_months": max(account_ages) if account_ages else np.nan,
        })

    # Apply feature extraction
    features_df = df.apply(row_feature_extraction, axis=1)

    # Combine with application_id
    result = pd.concat([df[["application_id"]].reset_index(drop=True), features_df.reset_index(drop=True)], axis=1)
    return result

In [126]:
df_payment_history_features = extract_normalized_credit_features(df_level1_flat)
df_payment_history_features.head()

,application_id,dpd_3m,dpd_6m,dpd_12m,dpd_24m,late_payments_12m_count,num_performing_accounts,num_nonperforming_accounts,has_written_off_account,num_open_accounts,num_new_accounts_6m,avg_utilization_ratio,total_amount_overdue,avg_account_age_months,max_account_age_months
0,9711360,4.0,4.0,5.0,36.0,8.0,7.0,0.0,1.0,3.0,0.0,0.188530,24042.07,112.285714,147.0
1,9714953,9.0,40.0,40.0,40.0,8.0,17.0,0.0,0.0,4.0,0.0,0.258610,NaN,80.058824,161.0
2,9714978,0.0,0.0,0.0,109.0,0.0,2.0,1.0,0.0,3.0,0.0,0.645756,12000.00,66.333333,72.0


## 5. Account Ratings

| Feature Name                | Description                             |
| --------------------------- | --------------------------------------- |
| `num_good_credit_cards`     | Number of good credit card accounts     |
| `num_bad_credit_cards`      | Number of bad credit card accounts      |
| `num_good_personal_loans`   | Number of good personal loan accounts   |
| `num_bad_personal_loans`    | Number of bad personal loan accounts    |
| `num_good_auto_loans`       | Number of good auto loans               |
| `num_bad_auto_loans`        | Number of bad auto loans                |
| `num_good_home_loans`       | Number of good home loans               |
| `num_bad_home_loans`        | Number of bad home loans                |
| `num_good_retail_accounts`  | Number of good retail accounts          |
| `num_bad_retail_accounts`   | Number of bad retail accounts           |
| `num_good_telecom_accounts` | Number of good telecom accounts         |
| `num_bad_telecom_accounts`  | Number of bad telecom accounts          |
| `num_good_study_loans`      | Number of good study loans              |
| `num_bad_study_loans`       | Number of bad study loans               |
| `num_good_joint_loans`      | Number of good joint loans              |
| `num_bad_joint_loans`       | Number of bad joint loans               |
| `num_good_other_accounts`   | Number of good other accounts           |
| `num_bad_other_accounts`    | Number of bad other accounts            |
| `total_good_accounts`       | Total good accounts across all products |
| `total_bad_accounts`        | Total bad accounts across all products  |
| `total_accounts`            | Sum of good + bad accounts              |
| `share_bad_accounts`        | Share of bad accounts = `bad / total`   |


In [81]:
def extract_account_rating_features(df, col='accountrating'):
    """
    Extract features from 'accountrating' dictionary column.

    Returns:
        pd.DataFrame with extracted variables.
    """

    product_mapping = {
        'creditcard': 'credit_cards',
        'personalloan': 'personal_loans',
        'autoloan': 'auto_loans',
        'homeloan': 'home_loans',
        'retailaccounts': 'retail_accounts',
        'telecomaccounts': 'telecom_accounts',
        'studyloan': 'study_loans',
        'jointloan': 'joint_loans',
        'otheraccounts': 'other_accounts'
    }

    def process_row(d):
        result = {}

        if not isinstance(d, dict):
            # Return all zeros
            base = {f'num_good_{v}': 0 for v in product_mapping.values()}
            base.update({f'num_bad_{v}': 0 for v in product_mapping.values()})
            base.update({
                'total_good_accounts': 0,
                'total_bad_accounts': 0,
                'total_accounts': 0,
                'share_bad_accounts': 0
            })
            return base

        total_good = 0
        total_bad = 0

        for key, val in d.items():
            try:
                count = int(val)
            except (ValueError, TypeError):
                count = 0

            for prefix, readable in product_mapping.items():
                if key.startswith(f'noof{prefix}'):
                    if key.endswith('good'):
                        result[f'num_good_{readable}'] = result.get(f'num_good_{readable}', 0) + count
                        total_good += count
                    elif key.endswith('bad'):
                        result[f'num_bad_{readable}'] = result.get(f'num_bad_{readable}', 0) + count
                        total_bad += count

        # Fill missing product vars with 0
        for v in product_mapping.values():
            result.setdefault(f'num_good_{v}', 0)
            result.setdefault(f'num_bad_{v}', 0)

        result['total_good_accounts'] = total_good
        result['total_bad_accounts'] = total_bad
        result['total_accounts'] = total_good + total_bad
        result['share_bad_accounts'] = round(total_bad / (total_good + total_bad), 3) if (total_good + total_bad) > 0 else 0

        return result

    features = df[col].apply(process_row).apply(pd.Series)
    return pd.concat([df['application_id'], features], axis=1)


In [82]:
df_accountrating_features = extract_account_rating_features(df_level1_flat, col='accountrating')
df_accountrating_features.head()

,application_id,num_bad_other_accounts,num_good_other_accounts,num_bad_retail_accounts,num_good_retail_accounts,num_bad_telecom_accounts,num_bad_auto_loans,num_good_auto_loans,num_bad_home_loans,num_good_telecom_accounts,num_good_home_loans,num_bad_joint_loans,num_bad_study_loans,num_bad_credit_cards,num_good_joint_loans,num_good_study_loans,num_good_credit_cards,num_bad_personal_loans,num_good_personal_loans,total_good_accounts,total_bad_accounts,total_accounts,share_bad_accounts
0,9711360,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,7.0,0.0,7.0,0.000
1,9714953,0.0,3.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,17.0,0.0,17.0,0.000
2,9714978,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,0.333


## 6. credit account summary

arrear_per_account = amount_in_arrear / total_accounts

debt_to_instalment_ratio = total_outstanding_debt / monthly_instalment_total

arrear_rate = total_accounts_in_arrear / total_accounts

has_judgement = 1 if num_judgements > 0 else 0

has_dishonoured = 1 if num_dishonoured > 0 else 0

In [129]:
def extract_credit_account_summary_features(df, col='creditaccountsummary'):
    """
    Extract financial features from the creditaccountsummary dictionary field.

    Returns:
        pd.DataFrame with added variables.
    """

    def parse_amount(val):
        try:
            return float(str(val).replace(',', '').replace(' ', '').replace('-', '0'))
        except:
            return 0.0

    def process_row(d):
        result = {}

        if not isinstance(d, dict):
            return {
                'credit_rating': 0,
                'amount_in_arrear': 0.0,
                'total_accounts': 0,
                'total_accounts_in_arrear': 0,
                'total_outstanding_debt': 0.0,
                'monthly_instalment_total': 0.0,
                'judgement_amount_total': 0.0,
                'num_judgements': 0,
                'num_dishonoured': 0,
                'num_good_condition_accounts': 0,
                'arrear_per_account': 0.0,
                'debt_to_instalment_ratio': 0.0,
                'arrear_rate': 0.0,
                'has_judgement': 0,
                'has_dishonoured': 0
            }

        # Raw values
        result['credit_rating'] = int(d.get('rating', 0))
        result['amount_in_arrear'] = parse_amount(d.get('amountarrear', 0))
        result['total_accounts'] = int(d.get('totalaccounts', 0))
        result['total_accounts_in_arrear'] = int(d.get('totalaccountarrear', 0))
        result['total_outstanding_debt'] = parse_amount(d.get('totaloutstandingdebt', 0))
        result['monthly_instalment_total'] = parse_amount(d.get('totalmonthlyinstalment', 0))
        result['judgement_amount_total'] = parse_amount(d.get('totaljudgementamount', 0))
        result['num_judgements'] = int(d.get('totalnumberofjudgement', 0))
        result['num_dishonoured'] = int(d.get('totalnumberofdishonoured', 0))
        result['num_good_condition_accounts'] = int(d.get('totalaccountingodcondition', 0))

        # Derived
        result['arrear_per_account'] = round(result['amount_in_arrear'] / result['total_accounts'], 2) if result['total_accounts'] else 0.0
        result['debt_to_instalment_ratio'] = round(result['total_outstanding_debt'] / result['monthly_instalment_total'], 2) if result['monthly_instalment_total'] else 0.0
        result['arrear_rate'] = round(result['total_accounts_in_arrear'] / result['total_accounts'], 2) if result['total_accounts'] else 0.0
        result['has_judgement'] = int(result['num_judgements'] > 0)
        result['has_dishonoured'] = int(result['num_dishonoured'] > 0)

        return result

    features = df[col].apply(process_row).apply(pd.Series)
    return pd.concat([df['application_id'], features], axis=1)

In [130]:
df_creditaccountsummary_features = extract_credit_account_summary_features(df_level1_flat, col='creditaccountsummary')
df_creditaccountsummary_features.head()


,application_id,credit_rating,amount_in_arrear,total_accounts,total_accounts_in_arrear,total_outstanding_debt,monthly_instalment_total,judgement_amount_total,num_judgements,num_dishonoured,num_good_condition_accounts,arrear_per_account,debt_to_instalment_ratio,arrear_rate,has_judgement,has_dishonoured
0,9711360,13.0,24041.0,7.0,2.0,105435.0,77404.0,0.0,0.0,0.0,0.0,3434.43,1.36,0.29,0.0,0.0
1,9714953,2.0,0.0,17.0,1.0,294770.0,132176.0,0.0,0.0,0.0,0.0,0.00,2.23,0.06,0.0,0.0
2,9714978,109.0,12000.0,3.0,1.0,110919.0,7000.0,0.0,0.0,0.0,0.0,4000.00,15.85,0.33,0.0,0.0


## 7. Credit Agreementy summmary


#### **Numeric Variables**
- `num_open_accounts` → Count of open accounts
- `num_written_off_accounts` → Number of accounts marked WrittenOff
- `num_new_accounts_6m` → New accounts opened in the last 6 months
- `total_current_balance` → Sum of current balances across accounts
- `total_opening_balance` → Sum of opening balances
- `total_amount_overdue` → Total overdue amount
- `max_amount_overdue` → Maximum overdue amount across accounts
- `avg_utilization_ratio` → Average utilization (current balance / opening balance)
- `avg_account_age_months` → Average age of accounts in months
- `max_account_age_months` → Maximum age of any account in months
- `num_personal_loans` → Number of personal loans
- `num_overdraft_accounts` → Number of overdraft accounts
- `num_installment_accounts` → Number of installment accounts
- `avg_instalment_amount` → Average EMI amount
- `overdue_to_balance_ratio` → Total overdue / Total current balance

#### **Flags (Binary Risk Indicators)**
- `has_written_off` → 1 if any account is WrittenOff
- `high_utilization_flag` → 1 if any utilization > 90%
- `high_overdue_flag` → 1 if any overdue > 50,000
- `multiple_written_off_flag` → 1 if more than 1 account is WrittenOff
- `recent_opening_spike_flag` → 1 if more than 2 accounts opened in last 6 months


In [95]:
import pandas as pd
import numpy as np
from datetime import datetime

def safe_float(val):
    try:
        return float(str(val).replace(",", "").strip())
    except:
        return np.nan

def extract_credit_agreement_summary_features(df, col='creditagreementsummary'):
    today = datetime.today()

    def process_credit_agreement(agreements):
        if not isinstance(agreements, list):
            return pd.Series({})

        num_open_accounts = 0
        num_written_off_accounts = 0
        num_new_accounts_6m = 0
        num_personal_loans = 0
        num_overdraft_accounts = 0
        num_installment_accounts = 0

        total_current_balance = 0
        total_opening_balance = 0
        total_amount_overdue = 0
        max_amount_overdue = 0
        utilisation_ratios = []
        instalment_amounts = []
        account_ages = []

        high_utilization_flag = 0
        high_overdue_flag = 0

        for acc in agreements:
            status = str(acc.get('accountstatus', '') or '')
            perf_status = str(acc.get('performancestatus', '') or '')
            indicator = str(acc.get('indicatordescription', '') or '')

            # Status-based features
            if status == 'Open':
                num_open_accounts += 1
            if status == 'WrittenOff':
                num_written_off_accounts += 1

            # Loan type classification
            if 'Personal' in indicator:
                num_personal_loans += 1
            if 'Overdraft' in indicator:
                num_overdraft_accounts += 1
            if 'Installment' in indicator:
                num_installment_accounts += 1

            # Balances & ratios
            current_balance = safe_float(acc.get('currentbalanceamt'))
            opening_balance = safe_float(acc.get('openingbalanceamt'))
            overdue = safe_float(acc.get('amountoverdue'))
            instalment_amount = safe_float(acc.get('instalmentamount'))

            total_current_balance += current_balance
            total_opening_balance += opening_balance
            total_amount_overdue += overdue
            max_amount_overdue = max(max_amount_overdue, overdue)

            if opening_balance > 0:
                util = current_balance / opening_balance
                utilisation_ratios.append(util)
                if util > 0.9:
                    high_utilization_flag = 1

            if overdue > 50000:
                high_overdue_flag = 1

            if instalment_amount > 0:
                instalment_amounts.append(instalment_amount)

            # Account age calculation
            date_opened_str = acc.get('dateaccountopened')
            if date_opened_str:
                try:
                    date_opened = datetime.strptime(date_opened_str, "%d/%m/%Y")
                    age_months = (today.year - date_opened.year) * 12 + (today.month - date_opened.month)
                    account_ages.append(age_months)
                    if age_months <= 6:
                        num_new_accounts_6m += 1
                except:
                    pass

        # Derived metrics
        avg_utilization_ratio = np.nanmean(utilisation_ratios) if utilisation_ratios else np.nan
        avg_instalment_amount = np.nanmean(instalment_amounts) if instalment_amounts else np.nan
        avg_account_age_months = np.nanmean(account_ages) if account_ages else np.nan
        max_account_age_months = max(account_ages) if account_ages else np.nan
        overdue_to_balance_ratio = total_amount_overdue / total_current_balance if total_current_balance > 0 else 0

        # Flags
        has_written_off = 1 if num_written_off_accounts > 0 else 0
        multiple_written_off_flag = 1 if num_written_off_accounts > 1 else 0
        recent_opening_spike_flag = 1 if num_new_accounts_6m > 2 else 0

        return pd.Series({
            "num_open_accounts": num_open_accounts,
            "num_written_off_accounts": num_written_off_accounts,
            "num_new_accounts_6m": num_new_accounts_6m,
            "total_current_balance": total_current_balance,
            "total_opening_balance": total_opening_balance,
            "total_amount_overdue": total_amount_overdue,
            "max_amount_overdue": max_amount_overdue,
            "avg_utilization_ratio": avg_utilization_ratio,
            "avg_account_age_months": avg_account_age_months,
            "max_account_age_months": max_account_age_months,
            "num_personal_loans": num_personal_loans,
            "num_overdraft_accounts": num_overdraft_accounts,
            "num_installment_accounts": num_installment_accounts,
            "avg_instalment_amount": avg_instalment_amount,
            "overdue_to_balance_ratio": overdue_to_balance_ratio,
            "has_written_off": has_written_off,
            "high_utilization_flag": high_utilization_flag,
            "high_overdue_flag": high_overdue_flag,
            "multiple_written_off_flag": multiple_written_off_flag,
            "recent_opening_spike_flag": recent_opening_spike_flag
        })

    features_df = df.apply(lambda row: process_credit_agreement(row[col]), axis=1)
    return pd.concat([df[['application_id']], features_df], axis=1)

In [96]:
df_creditagreementsummary_features = extract_credit_agreement_summary_features(df_level1_flat, col='creditagreementsummary')
df_creditagreementsummary_features.head()

,application_id,num_open_accounts,num_written_off_accounts,num_new_accounts_6m,total_current_balance,total_opening_balance,total_amount_overdue,max_amount_overdue,avg_utilization_ratio,avg_account_age_months,max_account_age_months,num_personal_loans,num_overdraft_accounts,num_installment_accounts,avg_instalment_amount,overdue_to_balance_ratio,has_written_off,high_utilization_flag,high_overdue_flag,multiple_written_off_flag,recent_opening_spike_flag
0,9711360,3.0,1.0,0.0,146825.79,1599179.98,24042.07,22441.39,0.188530,112.285714,147.0,3.0,2.0,1.0,19351.120000,0.163746,1.0,0.0,0.0,0.0,0.0
1,9714953,4.0,0.0,0.0,294770.06,621600.00,NaN,0.00,0.258610,80.058824,161.0,13.0,1.0,1.0,8811.703333,NaN,0.0,1.0,0.0,0.0,0.0
2,9714978,3.0,0.0,0.0,110919.00,1090936.00,12000.00,12000.00,0.645756,66.333333,72.0,3.0,1.0,0.0,7000.000000,0.108187,0.0,1.0,0.0,0.0,0.0


## 8.Employment history

| Feature Name                   | Description                                                          |
| ------------------------------ | -------------------------------------------------------------------- |
| `num_unique_occupations`       | Count of unique occupations across all jobs                          |
| `num_military_or_defense_jobs` | Number of jobs in Army, Navy, Military, or Defense sectors           |
| `years_since_last_update`      | Years since the last employment record update                        |
| `current_occupation`           | Occupation from the most recent employment record                    |
| `is_government_job`            | 1 if current occupation is a government job (e.g., Public Servant)   |
| `is_high_stability_job`        | 1 if government or defense-related job (stable employment indicator) |


In [99]:
def extract_employment_history_features(df, col='employmenthistory'):
    today = datetime.today()
    
    def process_employment(history):
        if not isinstance(history, list):
            return pd.Series({})

        num_employers = len(history)
        unique_occupations = set()
        num_military_or_defense_jobs = 0
        last_update_date = None
        current_occupation = None

        for entry in history:
            occupation = str(entry.get('occupation', '') or '').strip().upper()
            employer = str(entry.get('employerdetail', '') or '').strip().upper()
            update_date_str = entry.get('updatedate') or entry.get('updateondate')

            if occupation:
                unique_occupations.add(occupation)
            if 'ARMY' in occupation or 'DEFENCE' in occupation or 'MILITARY' in occupation or 'NAVY' in employer:
                num_military_or_defense_jobs += 1

            if update_date_str:
                try:
                    update_date = datetime.strptime(update_date_str, "%d/%m/%Y")
                    if (last_update_date is None) or (update_date > last_update_date):
                        last_update_date = update_date
                except:
                    pass

        # Current occupation → last record in the list
        if history and isinstance(history[-1], dict):
            current_occupation = str(history[-1].get('occupation', '') or '').upper()

        years_since_last_update = (today - last_update_date).days / 365 if last_update_date else np.nan
        is_government_job = 1 if 'PUBLIC SERVANTS' in current_occupation else 0
        is_high_stability_job = 1 if is_government_job or num_military_or_defense_jobs > 0 else 0

        return pd.Series({
            # "num_employers": num_employers,
            "num_unique_occupations": len(unique_occupations),
            "num_military_or_defense_jobs": num_military_or_defense_jobs,
            "years_since_last_update": years_since_last_update,
            "current_occupation": current_occupation,
            "is_government_job": is_government_job,
            "is_high_stability_job": is_high_stability_job
        })

    features_df = df.apply(lambda row: process_employment(row[col]), axis=1)
    return pd.concat([df[['application_id']], features_df], axis=1)

In [100]:
df_employmenthistory_features = extract_employment_history_features(df_level1_flat, col='employmenthistory')
df_employmenthistory_features.head()

,application_id,num_unique_occupations,num_military_or_defense_jobs,years_since_last_update,current_occupation,is_government_job,is_high_stability_job
0,9711360,2,1,8.884932,PUBLIC SERVANTS,1,1
1,9714953,2,0,10.706849,DOCTOR,0,0
2,9714978,1,0,NaN,STUDENT,0,0


## 9. Enquiry details

From Latest Enquiry (enquirydetails)
latest_matching_rate → Matching rate from the most recent enquiry (numeric)

latest_product_id → Product ID from the latest enquiry (categorical)

has_recent_enquiry → Flag (1/0) if any enquiry exists

From Enquiry History (enquiryhistorytop)
num_total_enquiries → Total number of enquiries made by the customer

num_enquiries_last_30d → Number of enquiries in the last 30 days

num_enquiries_last_90d → Number of enquiries in the last 90 days

days_since_last_enquiry → Days since the most recent enquiry

num_unique_subscribers → Number of unique lenders/banks making enquiries (proxy for credit shopping)

recent_enquiry_flag → 1 if there was an enquiry in the last 30 days, else 0

high_enquiry_risk_bucket → Categorized as:

Low → ≤ 3 enquiries in last 90 days

Medium → 4–6 enquiries in last 90 days

High → > 6 enquiries in last 90 days

In [111]:
def extract_enquiry_features(df, enquiry_details_col='enquirydetails', enquiry_history_col='enquiryhistorytop'):
    today = datetime.today()

    def process_enquiry_features(row):
        enquiry_details = row.get(enquiry_details_col, {})
        enquiry_history = row.get(enquiry_history_col, [])

        # ---- From enquirydetails ----
        latest_matching_rate = float(enquiry_details.get('matchingrate', 0)) if enquiry_details else 0
        latest_product_id = enquiry_details.get('productid', None)
        has_recent_enquiry = 1 if enquiry_details else 0

        # ---- From enquiryhistorytop ----
        if not enquiry_history:
            return pd.Series({
                "latest_matching_rate": latest_matching_rate,
                "latest_product_id": latest_product_id,
                "has_recent_enquiry": has_recent_enquiry,
                "num_total_enquiries": 0,
                "num_enquiries_last_30d": 0,
                "num_enquiries_last_90d": 0,
                "days_since_last_enquiry": np.nan,
                "num_unique_subscribers": 0,
                "recent_enquiry_flag": 0,
                "high_enquiry_risk_bucket": "Low"
            })

        dates = []
        subscribers = []
        for e in enquiry_history:
            date_str = e.get('daterequested')
            if date_str:
                try:
                    date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")
                    dates.append(date_obj)
                except:
                    pass
            subscribers.append(e.get('subscribername'))

        num_total = len(enquiry_history)
        unique_subs = len(set(subscribers))

        if dates:
            last_date = max(dates)
            days_since_last = (today - last_date).days
        else:
            days_since_last = np.nan

        num_last_30d = sum((today - d).days <= 30 for d in dates)
        num_last_90d = sum((today - d).days <= 90 for d in dates)

        # Flags & Risk bucket
        recent_flag = 1 if num_last_30d > 0 else 0
        if num_last_90d <= 3:
            bucket = "Low"
        elif num_last_90d <= 6:
            bucket = "Medium"
        else:
            bucket = "High"

        return pd.Series({
            "latest_matching_rate": latest_matching_rate,
            "latest_product_id": latest_product_id,
            "has_recent_enquiry": has_recent_enquiry,
            "num_total_enquiries": num_total,
            "num_enquiries_last_30d": num_last_30d,
            "num_enquiries_last_90d": num_last_90d,
            "days_since_last_enquiry": days_since_last,
            "num_unique_subscribers": unique_subs,
            "recent_enquiry_flag": recent_flag,
            "high_enquiry_risk_bucket": bucket
        })

    features_df = df.apply(process_enquiry_features, axis=1)
    return pd.concat([df[['application_id']], features_df], axis=1)

In [112]:
df_enquirydetails_features = extract_enquiry_features(df_level1_flat)
df_enquirydetails_features.head()

,application_id,latest_matching_rate,latest_product_id,has_recent_enquiry,num_total_enquiries,num_enquiries_last_30d,num_enquiries_last_90d,days_since_last_enquiry,num_unique_subscribers,recent_enquiry_flag,high_enquiry_risk_bucket
0,9711360,90.0,45,1,15,0,0,1822,2,0,Low
1,9714953,90.0,45,1,10,0,0,1829,4,0,Low
2,9714978,90.0,45,1,10,0,0,1829,3,0,Low


## 10. Telephone History

num_unique_home_phones → Number of unique home telephone numbers

num_unique_mobile_phones → Number of unique mobile numbers

num_phone_updates → Total number of phone update records

days_since_last_phone_update → Days since the most recent phone update

days_between_first_and_last_update → Time gap between first and last update

has_multiple_mobile_numbers → 1 if more than one unique mobile number exists

has_recent_update_flag → 1 if a phone update occurred within the last 90 days

In [115]:
def extract_telephone_history_features(df, col='telephonehistory'):
    
    today = datetime.today()
    
    features = []
    
    for idx, row in df.iterrows():
        tel_history = row.get(col, [])
        
        home_numbers = set()
        mobile_numbers = set()
        update_dates = []
        
        for record in tel_history:
            home = record.get('hometelephonenumber', None)
            mobile = record.get('mobiletelephonenumber', None)
            update_date_str = record.get('homenoupdatedondate', None)
            
            if home and home != 'XXX':
                home_numbers.add(home)
            if mobile and mobile != 'XXX':
                mobile_numbers.add(mobile)
            
            if update_date_str and update_date_str != 'XXX':
                try:
                    update_date = datetime.strptime(update_date_str, '%d/%m/%Y')
                    update_dates.append(update_date)
                except:
                    pass
        
        # Compute numeric features
        num_unique_home_phones = len(home_numbers)
        num_unique_mobile_phones = len(mobile_numbers)
        num_phone_updates = len(update_dates)
        
        # Date-based features
        days_since_last_phone_update = None
        days_between_first_and_last_update = None
        if update_dates:
            last_update = max(update_dates)
            first_update = min(update_dates)
            days_since_last_phone_update = (today - last_update).days
            days_between_first_and_last_update = (last_update - first_update).days
        
        # Flags
        has_multiple_mobile_numbers = 1 if num_unique_mobile_phones > 1 else 0
        has_recent_update_flag = 1 if days_since_last_phone_update is not None and days_since_last_phone_update <= 90 else 0
        
        features.append({
            'num_unique_home_phones': num_unique_home_phones,
            'num_unique_mobile_phones': num_unique_mobile_phones,
            'num_phone_updates': num_phone_updates,
            'days_since_last_phone_update': days_since_last_phone_update,
            'days_between_first_and_last_update': days_between_first_and_last_update,
            'has_multiple_mobile_numbers': has_multiple_mobile_numbers,
            'has_recent_update_flag': has_recent_update_flag
        })
    
    features_df = pd.DataFrame(features)
    return pd.concat([df[['application_id']], features_df], axis=1)

In [116]:
df_telephone_features = extract_telephone_history_features(df_level1_flat, col='telephonehistory')
df_telephone_features.head()

,application_id,num_unique_home_phones,num_unique_mobile_phones,num_phone_updates,days_since_last_phone_update,days_between_first_and_last_update,has_multiple_mobile_numbers,has_recent_update_flag
0,9711360,0,0,3,3404.0,225.0,0,0
1,9714953,0,0,3,3404.0,500.0,0,0
2,9714978,0,0,0,NaN,NaN,0,0


## Merge All Variables

In [135]:
def extract_all_credit_features(df):
    """
    Combine all feature extraction functions into a single function.
    Input: df (DataFrame with all raw JSON fields)
    Output: DataFrame with application_id and all engineered features.
    """

    # 1. Account Monthly Payment History
    df_payment_history_features = extract_normalized_credit_features(df_level1_flat)

    # 2. Credit Agreement Summary
    credit_agreement_features = extract_credit_agreement_summary_features(df, col='creditagreementsummary')

    # 3. Account Rating
    account_rating_features = extract_account_rating_features(df, col='accountrating')

    # 4. Credit Account Summary
    credit_account_summary_features = extract_credit_account_summary_features(df, col='creditaccountsummary')

    # 5. Employment History
    employment_history_features = extract_employment_history_features(df, col='employmenthistory')

    # 6. Enquiry Details & History
    enquiry_features = extract_enquiry_features(df) #details_col='enquirydetails', history_col='enquiryhistorytop')

    # 7. Telephone History
    telephone_history_features = extract_telephone_history_features(df, col='telephonehistory')

    # 8. Delinquency Information
    delinquency_features = extract_delinquency_features(df, col='deliquencyinformation')

    # 9. Personal Details Summary
    personal_details_features = extract_personal_details_features(df, col='personaldetailssummary')

    # 10. Guarantor Details
    guarantor_features = extract_guarantor_features(df)

    # Combine all
    all_features = [
        df_payment_history_features,
        credit_agreement_features,
        account_rating_features,
        credit_account_summary_features,
        employment_history_features,
        enquiry_features,
        telephone_history_features,
        delinquency_features,
        personal_details_features,
        guarantor_features
    ]

    # Merge all on application_id
    final_features = all_features[0]
    for f in all_features[1:]:
        final_features = final_features.merge(f, on='application_id', how='left')

    return final_features

In [137]:
final_features_df = extract_all_credit_features(df_level1_flat)
print(final_features_df.shape)
final_features_df.head()

(3, 112)


,application_id,dpd_3m,dpd_6m,dpd_12m,dpd_24m,late_payments_12m_count,num_performing_accounts,num_nonperforming_accounts,has_written_off_account,num_open_accounts_x,num_new_accounts_6m_x,avg_utilization_ratio_x,total_amount_overdue_x,avg_account_age_months_x,max_account_age_months_x,num_open_accounts_y,num_written_off_accounts,num_new_accounts_6m_y,total_current_balance,total_opening_balance,total_amount_overdue_y,max_amount_overdue,avg_utilization_ratio_y,avg_account_age_months_y,max_account_age_months_y,num_personal_loans,num_overdraft_accounts,num_installment_accounts,avg_instalment_amount,overdue_to_balance_ratio,has_written_off,high_utilization_flag,high_overdue_flag,multiple_written_off_flag,recent_opening_spike_flag,num_bad_other_accounts,num_good_other_accounts,num_bad_retail_accounts,num_good_retail_accounts,num_bad_telecom_accounts,num_bad_auto_loans,num_good_auto_loans,num_bad_home_loans,num_good_telecom_accounts,num_good_home_loans,num_bad_joint_loans,num_bad_study_loans,num_bad_credit_cards,num_good_joint_loans,num_good_study_loans,num_good_credit_cards,num_bad_personal_loans,num_good_personal_loans,total_good_accounts,total_bad_accounts,total_accounts_x,share_bad_accounts,credit_rating,amount_in_arrear,total_accounts_y,total_accounts_in_arrear,total_outstanding_debt,monthly_instalment_total,judgement_amount_total,num_judgements,num_dishonoured,num_good_condition_accounts,arrear_per_account,debt_to_instalment_ratio,arrear_rate,has_judgement,has_dishonoured,num_unique_occupations,num_military_or_defense_jobs,years_since_last_update,current_occupation,is_government_job,is_high_stability_job,latest_matching_rate,latest_product_id,has_recent_enquiry,num_total_enquiries,num_enquiries_last_30d,num_enquiries_last_90d,days_since_last_enquiry,num_unique_subscribers,recent_enquiry_flag,high_enquiry_risk_bucket,num_unique_home_phones,num_unique_mobile_phones,num_phone_updates,days_since_last_phone_update,days_between_first_and_last_update,has_multiple_mobile_numbers,has_recent_update_flag,total_delinquency_accounts,months_in_arrears,latest_delinquency_period,unique_delinquency_subscribers,has_delinquency_flag,severe_delinquency_flag,recent_delinquency_flag,gender,dependants,nationality,employerdetail,residential_state,age,guarantordateofbirth,guarantor_age,guarantor_count,has_guarantor
0,9711360,4.0,4.0,5.0,36.0,8.0,7.0,0.0,1.0,3.0,0.0,0.188530,24042.07,112.285714,147.0,3.0,1.0,0.0,146825.79,1599179.98,24042.07,22441.39,0.188530,112.285714,147.0,3.0,2.0,1.0,19351.120000,0.163746,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,7.0,0.0,7.0,0.000,13.0,24041.0,7.0,2.0,105435.0,77404.0,0.0,0.0,0.0,0.0,3434.43,1.36,0.29,0.0,0.0,2,1,8.884932,PUBLIC SERVANTS,1,1,90.0,45,1,15,0,0,1822,2,0,Low,0,0,3,3404.0,225.0,0,0,1,13,20140930,1,1,1,1,Male,0,Nigeria,ALL MILITARY STAFFS,KADUNA STATE NIGERIA,34,NaT,NaN,0,0
1,9714953,9.0,40.0,40.0,40.0,8.0,17.0,0.0,0.0,4.0,0.0,0.258610,NaN,80.058824,161.0,4.0,0.0,0.0,294770.06,621600.00,NaN,0.00,0.258610,80.058824,161.0,13.0,1.0,1.0,8811.703333,NaN,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,17.0,0.0,17.0,0.000,2.0,0.0,17.0,1.0,294770.0,132176.0,0.0,0.0,0.0,0.0,0.00,2.23,0.06,0.0,0.0,2,0,10.706849,DOCTOR,0,0,90.0,45,1,10,0,0,1829,4,0,Low,0,0,3,3404.0,500.0,0,0,0,2,20170430,1,0,0,1,Female,0,Nigeria,None,UMUAHIA,40,NaT,NaN,0,0
2,9714978,0.0,0.0,0.0,109.0,0.0,2.0,1.0,0.0,3.0,0.0,0.645756,12000.00,66.333333,72.0,3.0,0.0,0.0,110919.00,1090936.00,12000.00,12000.00,0.645756,66.333333,72.0,3.0,1.0,0.0,7000.000000,0.108187,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,0.333,109.0,12000.0,3.0,1.0,110919.0,7000.0,0.0,0.0,0.0,0.0,4000.00,15.85,0.33,0.0,0.0,1,0,NaN,STUDENT,0,0,90.0,45,1,10,0,0,1829,3,0,Low,0,0,0,NaN,NaN,0,0,1,109,20200630,1,1,1,1,Female,0,Nigeria,None,Nigeria,42,NaT,NaN,0,0
